#Run cross-group analyses that lyman doesn't suppport

In [7]:
#os and i/o
import os
import numpy as np
import glob
from os.path import abspath

#scientific computing
import matplotlib
# import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats, optimize
from pandas import DataFrame, Series
import seaborn as sns
import random as rd
import scipy.stats
import multiprocessing
##nipype
import nibabel as nib

%matplotlib inline

In [22]:
#preliminary housekeeping
home_dir = '/data/home/iballard/mag/'
subj_file = home_dir + 'subjects.txt'
sub_list = list(np.loadtxt(subj_file,'string'))
print sub_list
os.chdir(home_dir)
exps = ['mag-maxSV']
runs = map(str,range(1,5))

['asu4', 'asu8', 'asu10', 'asu11', 'asu12', 'asu13', 'subject03', 'subject04', 'subject05', 'subject06', 'subject08', 'subject09', 'subject10', 'subject11', 'subject12', 'subject13', 'subject14', 'subject20', 'subject21']


In [23]:
group_dir = home_dir + 'analysis/mag-maxSV/randomise/'
if not os.path.exists(group_dir):
    os.mkdir(group_dir)

# Merge files

In [42]:
#merge the 3d files from ffx analysis into concatenated 4d files
images = ['cope']
ims_to_files = {'cope':'cope1.nii.gz'}
contrasts = glob.glob(home_dir + 'analysis/mag-maxSV/group/mni/*')
contrasts = [x.split('/')[-1] for x in contrasts]
for contrast in contrasts:
    for f in images:
        out_dir = group_dir + contrast
        if not os.path.exists(out_dir):
            os.mkdir(out_dir)

        #add files to merge command
        out_f = out_dir + '/' + f + '_merged.nii.gz'
        if os.path.exists(out_f):
            os.remove(out_f)
            
        cmd_str = ['fslmerge','-t',out_f]

        for exp in exps: #order matters now
            for sub in sub_list:
                sub_dir = home_dir + '/analysis/' + exp + '/' + sub 
                sub_f = sub_dir + '/ffx/mni/smoothed/' + contrast + '/' + ims_to_files[f]
                cmd_str.append(sub_f)
        cmd_str = ' '.join(cmd_str)
        os.system(cmd_str)

# Do fitting with randomise

In [44]:
#do fitting with randomise
#input image files
def randomise(contrast):
    im_dir = group_dir + contrast + '/'
        
    stat_file = im_dir + 'cope_merged.nii.gz'
    mask_file = home_dir + '/analysis/' + exp + '/group/mni/' + contrast + '/mask.nii.gz'
    cmd = ['randomise','-i',stat_file,'-o',im_dir,'-m', mask_file,
           '-T','-1','-n','1000','--uncorrp']
    cmd = ' '.join(cmd)
    os.system(cmd)
    
    cmd = ['randomise','-i',stat_file,'-o',im_dir,'-m', mask_file,
           '-C','1.7','-1','-n','1000']
    cmd = ' '.join(cmd)
#     os.system(cmd)

In [46]:
#do fitting with randomise
#input image files
def fdr(contrast):
    im_dir = group_dir + contrast + '/'
        
    stat_file = im_dir + 'cope_merged.nii.gz'
    mask_file = home_dir + '/analysis/' + exp + '/group/mni/' + contrast + '/mask.nii.gz'
    
    cmd = ['fdr','-i',im_dir + '_tfce_p_tstat1','--oneminusp','-m',
           mask_file,'-q', '0.05', '--othresh=' + im_dir + 'fdr_tstat1']
    cmd = ' '.join(cmd)
    os.system(cmd)


In [47]:
# pool = multiprocessing.Pool(processes = len(contrasts))
# pool.map(randomise,contrasts)
# pool.terminate()
# pool.join()

In [48]:
pool = multiprocessing.Pool(processes = len(contrasts))
pool.map(fdr,contrasts)
pool.terminate()
pool.join()